In [1]:
# Import Libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
nabil = pd.read_csv("nabil_1yr.csv")
nabil.head()

,SN,Close
0,1,1450.0
1,2,1480.0
2,3,1499.0
3,4,1524.0
4,5,1485.0


In [12]:
train_data = nabil[['Close']]
train_data

,Close
0,1450.0
1,1480.0
2,1499.0
3,1524.0
4,1485.0
...,...
359,737.0
360,729.0
361,720.0
362,770.0


### Normalizing the Data:

In [13]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_training_set = scaler.fit_transform(train_data)

scaled_training_set

array([[0.78242229],
       [0.81457663],
       [0.83494105],
       [0.86173633],
       [0.81993569],
       [0.81993569],
       [0.78510182],
       [0.75026795],
       [0.76098607],
       [0.78242229],
       [0.79314041],
       [0.69249732],
       [0.75133976],
       [0.78553055],
       [0.78971061],
       [0.80921758],
       [0.81993569],
       [0.80814577],
       [0.8488746 ],
       [0.8619507 ],
       [0.8585209 ],
       [0.86387996],
       [0.86066452],
       [0.87674169],
       [0.88424437],
       [0.87566988],
       [0.88531618],
       [0.88317256],
       [0.886388  ],
       [0.8960343 ],
       [0.89924973],
       [0.87888532],
       [0.88745981],
       [0.88960343],
       [0.92068596],
       [0.95894962],
       [1.        ],
       [0.95176849],
       [0.96463023],
       [0.92175777],
       [0.94105038],
       [0.94962487],
       [0.90996785],
       [0.83279743],
       [0.80482315],
       [0.80493033],
       [0.79314041],
       [0.762

### Creating X_train and Y_train data:

In [14]:
x_train = []
y_train = []
for i in range(60, 364):
    x_train.append(scaled_training_set[i-60:i, 0])
    y_train.append(scaled_training_set[i, 0])
    
x_train = np.array(x_train)
y_train = np.array(y_train)

In [15]:
print(x_train.shape)
print(y_train.shape)

(304, 60)
(304,)


In [16]:
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_train.shape

(304, 60, 1)

### Building the model:

In [17]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Dropout

In [18]:
regressor = Sequential()

regressor.add(LSTM(units=50, return_sequences=True, input_shape = (x_train.shape[1], 1)))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units=50))
regressor.add(Dropout(0.2))

regressor.add(Dense(units=1))

### Training the model:

In [19]:
regressor.compile(optimizer='adam', loss='mean_squared_error')
regressor.fit(x_train, y_train, epochs=100, batch_size=32)

Epoch 1/100
10/10 [==============================] - 15s 48ms/step - loss: 0.1166
Epoch 2/100
10/10 [==============================] - 0s 35ms/step - loss: 0.0300
Epoch 3/100
10/10 [==============================] - 0s 33ms/step - loss: 0.0221
Epoch 4/100
10/10 [==============================] - 0s 29ms/step - loss: 0.0159
Epoch 5/100
10/10 [==============================] - 0s 32ms/step - loss: 0.0116: 0s - loss: 0.0
Epoch 6/100
10/10 [==============================] - 0s 34ms/step - loss: 0.0137
Epoch 7/100
10/10 [==============================] - 0s 34ms/step - loss: 0.0123
Epoch 8/100
10/10 [==============================] - 0s 29ms/step - loss: 0.0134
Epoch 9/100
10/10 [==============================] - 0s 34ms/step - loss: 0.0120
Epoch 10/100
10/10 [==============================] - 0s 32ms/step - loss: 0.0108
Epoch 11/100
10/10 [==============================] - 0s 33ms/step - loss: 0.0117
Epoch 12/100
10/10 [==============================] - 0s 37ms/step - loss: 0.0100
Epoch 13

10/10 [==============================] - 0s 31ms/step - loss: 0.0045


## Evaluating:

In [20]:
test_df = pd.read_csv("nabil_1yr.csv")
actual_stock_price = test_df[['Close']]

### Preparing Input:

In [22]:
merged_df = pd.concat((train_data['Close'], test_df['Close']), axis=0)
inputs = merged_df[len(merged_df) - len(merged_df)-60:].values

inputs = inputs.reshape(-1, 1)
inputs = scaler.transform(inputs)

x_test = []
for i in range(60, 80):
    x_test.append(inputs[i-60:i, 0])
x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

C:\Users\ASUS\anaconda3\envs\tf_gpu\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
C:\Users\ASUS\AppData\Local\Temp/ipykernel_16932/4054883090.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_test = np.array(x_test)


IndexError: tuple index out of range